In [2]:
import pandas as pd
import numpy as np

In [14]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

# importar os pacotes necessários para os algoritmos de classificação
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [97]:
df = pd.read_csv('afastamentos-nomes-sexo.csv', index_col=0)

In [98]:
df.drop(columns=['REV'], inplace=True)

In [99]:
df.head(10)

,QTDE,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
PNOME,,,,,,,,,,,,,,,,,
MARIA,2937,F,5,A,I,R,A,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JOSE,2766,M,4,E,S,O,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANTONIO,1257,M,7,O,I,N,O,T,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FRANCISCO,1014,M,9,O,C,S,I,C,N,A,R,F,NaN,NaN,NaN,NaN,NaN
JOAO,993,M,4,O,A,O,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CARLOS,979,M,6,S,O,L,R,A,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANA,938,F,3,A,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAULO,907,M,5,O,L,U,A,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LUIZ,829,M,4,Z,I,U,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df['SEXO'] = df['SEXO'].map({'F': 0, 'M': 1})
df.loc[df['SEXO'].isnull(), 'SEXO'] = 2
df['SEXO'] = df['SEXO'].astype('uint8')

In [101]:
for i in range(14):
    col = 'L' + str(i)
    df.loc[df[col].isnull(), col] = ' '
    df[col] = df[col].apply(
        lambda x: ord(x) - 64 if x != ' ' else 0
    ).astype('uint8')

In [102]:
df.dtypes

QTDE      int64
SEXO      uint8
LETRAS    int64
L0        uint8
L1        uint8
L2        uint8
L3        uint8
L4        uint8
L5        uint8
L6        uint8
L7        uint8
L8        uint8
L9        uint8
L10       uint8
L11       uint8
L12       uint8
L13       uint8
dtype: object

In [103]:
df.head(10)

,QTDE,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
PNOME,,,,,,,,,,,,,,,,,
MARIA,2937,0,5,1,9,18,1,13,0,0,0,0,0,0,0,0,0
JOSE,2766,1,4,5,19,15,10,0,0,0,0,0,0,0,0,0,0
ANTONIO,1257,1,7,15,9,14,15,20,14,1,0,0,0,0,0,0,0
FRANCISCO,1014,1,9,15,3,19,9,3,14,1,18,6,0,0,0,0,0
JOAO,993,1,4,15,1,15,10,0,0,0,0,0,0,0,0,0,0
CARLOS,979,1,6,19,15,12,18,1,3,0,0,0,0,0,0,0,0
ANA,938,0,3,1,14,1,0,0,0,0,0,0,0,0,0,0,0
PAULO,907,1,5,15,12,21,1,16,0,0,0,0,0,0,0,0,0
LUIZ,829,1,4,26,9,21,12,0,0,0,0,0,0,0,0,0,0


In [104]:
df.describe()

,QTDE,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
count,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000,9726.000000
mean,6.571561,0.655871,6.884536,10.076599,10.846185,10.888649,9.988073,10.048221,8.371273,6.084516,3.523031,1.568785,0.475735,0.125334,0.035986,0.006992,0.001439
std,53.367211,0.661322,1.605880,7.393167,6.155816,6.477221,6.546434,6.997042,7.450970,7.371567,6.253763,4.492524,2.534731,1.327666,0.761098,0.270109,0.101394
min,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,6.000000,1.000000,5.000000,5.000000,5.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,7.000000,12.000000,12.000000,12.000000,9.000000,9.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,8.000000,15.000000,15.000000,14.000000,14.000000,15.000000,14.000000,12.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2937.000000,2.000000,14.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,25.000000,23.000000,25.000000,18.000000,8.000000


In [105]:
# definir dados de entrada

X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (9726, 16) (9726,)


In [106]:
X.head()

,QTDE,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
PNOME,,,,,,,,,,,,,,,,
MARIA,2937,5,1,9,18,1,13,0,0,0,0,0,0,0,0,0
JOSE,2766,4,5,19,15,10,0,0,0,0,0,0,0,0,0,0
ANTONIO,1257,7,15,9,14,15,20,14,1,0,0,0,0,0,0,0
FRANCISCO,1014,9,15,3,19,9,3,14,1,18,6,0,0,0,0,0
JOAO,993,4,15,1,15,10,0,0,0,0,0,0,0,0,0,0


In [107]:
y.head()

PNOME
MARIA        0
JOSE         1
ANTONIO      1
FRANCISCO    1
JOAO         1
Name: SEXO, dtype: uint8

In [108]:
from datetime import datetime

models = []

def evaluate_model_cv(name, model, X=X, y=y):
    start = datetime.now()
    models.append((name, model))
    kfold = KFold(n_splits=10, random_state=42)
    results = cross_val_score(model, X, y, cv=kfold,
                              scoring='accuracy', verbose=1)
    end = datetime.now()
    elapsed = int((end - start).total_seconds() * 1000)
    score = results.mean() * 100
    stddev = results.std() * 100
    print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
          (score, stddev, elapsed))
    return score, stddev, elapsed

In [109]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):
  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  kfold = KFold(n_splits=10, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, n_iter=20, scoring='accuracy', cv=kfold, verbose=1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [4]:
#'var_smoothing': 
np.logspace(0,-9, num=10)

array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09])

In [110]:
# A) Logistic Regression
model = LogisticRegression(random_state=42, solver='lbfgs', multi_class='auto', max_iter=500, C=100)
#TODO: testar LogisticRegression(multi_class='multinomial', solver='newton-cg')
evaluate_model_cv(model)

params = {'solver':['liblinear', 'lbfgs'], 'C':np.logspace(-3,3,7)}
#fine_tune_model(model, params)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs fail

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) 
Cross-Validation Score: 84.45 (+/- 3.11) [116940 ms]


/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.9min finished


NameError: name 'np' is not defined

In [112]:
# B) Decision Tree
model = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=11)
evaluate_model_cv(model)

params = {'criterion':['gini','entropy'], 'max_depth':[3,5,7,11]}
#fine_tune_model(model, params)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=11,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best') 
Cross-Validation Score: 96.02 (+/- 2.11) [ 2423 ms]


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.4s finished


In [113]:
# C) K-Nearest Neighbours
model = KNeighborsClassifier(n_neighbors=1)
evaluate_model_cv(model)

params = {'n_neighbors':[1, 3, 5, 7, 9]}
#fine_tune_model(model, params)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform') 
Cross-Validation Score: 85.25 (+/- 1.92) [22694 ms]


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   22.7s finished


In [114]:
# D) Support Vector Machine (SVM)
model = SVC(random_state=42, C=10, gamma=0.1, kernel='rbf')
# SVC(kernel='linear', C=1)
evaluate_model_cv(model)

params = {'C':[0.001, 0.01, 0.1, 1, 10, 100], 'gamma':[0.001, 0.01, 0.1, 1, 10, 100], 'kernel':['linear', 'rbf']}
#fine_tune_model(model, params)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [ ]:
# E) Random Forest
model = RandomForestClassifier(random_state=42, max_features='auto', n_estimators=10)
evaluate_model_cv(model)

params = {'n_estimators':[10, 50, 100, 500], 'max_features':['auto', 'sqrt', 'log2']}
#fine_tune_model(model, params)

In [ ]:
# F) Stochastic Gradient Descent (SGD)
model = SGDClassifier(random_state=42, max_iter=100, tol=0.1)
evaluate_model_cv(model)

params = {'max_iter':[100, 200, 350, 500, 1000], 'tol':[0.1, 0.01]}
#fine_tune_model(model, params)

In [ ]:
# G) Perceptron
model = Perceptron(random_state=42, max_iter=100, tol=0.01)
# Perceptron(eta0=1, random_state=1)
evaluate_model_cv(model)

params = {'max_iter':[100, 200, 350, 500, 1000], 'tol':[0.1, 0.01, 0.001]}
#fine_tune_model(model, params)

In [ ]:
# H) Naïve Bayes
model = GaussianNB(priors=None, var_smoothing=1e-08)
evaluate_model_cv(model)

params = {'priors': [None], 'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]}
#fine_tune_model(model, params)

In [ ]:
# I) Linear SVM
model = LinearSVC(random_state=42, max_iter=1000, C=10)
evaluate_model_cv(model)

params = {'C':[0.001, 0.01, 0.1, 1, 10, 100]}
#fine_tune_model(model, params)

In [ ]:
# J) Ada Boost
model = AdaBoostClassifier(DecisionTreeClassifier(random_state=42), n_estimators=5)
evaluate_model_cv(model)

params = {'n_estimators':[1,3,5,7,11]}
#fine_tune_model(model, params)

In [ ]:
# K) Gradient Boosting
model = GradientBoostingClassifier(random_state=42, max_depth=3)
evaluate_model_cv(model)

'''
params = {
    "learning_rate":[0.01, 0.05, 0.1],
    "max_depth":[3, 5, 7],
    "max_features":["log2", "sqrt"],
    "criterion":["friedman_mse", "mae"],
    "subsample":[0.5, 0.75, 1.0],
}
'''

params = {'max_depth':[3, 5, 7]}
#fine_tune_model(model, params)

In [ ]:
# M) Multi-Layer Perceptron (MLP)
model = MLPClassifier(random_state=42, solver='lbfgs', alpha=0.1, hidden_layer_sizes=(15,))
evaluate_model_cv(model)

params = {'alpha':[1,0.1,0.01,0.001,0.0001,0]}
#fine_tune_model(model, params)

In [ ]:
# N) Linear Discriminant Analysis (LDA)
model = LinearDiscriminantAnalysis(solver='svd')
evaluate_model_cv(model)

params = {'solver':['svd', 'lsqr']} #, 'eigen']}
#fine_tune_model(model, params)

In [ ]:
models = []
models.append(('LR', LogisticRegression(random_state=42, solver='lbfgs', multi_class='auto', max_iter=500, C=100)))
models.append(('DT', DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=11)))
models.append(('KNN', KNeighborsClassifier(n_neighbors=1)))
models.append(('SVM', SVC(random_state=42, C=10, gamma=0.1, kernel='rbf')))
models.append(('RF', RandomForestClassifier(random_state=42, max_features='auto', n_estimators=10)))
models.append(('SGD', SGDClassifier(random_state=42, max_iter=100, tol=0.1)))
models.append(('NN', Perceptron(random_state=42, max_iter=100, tol=0.01)))
models.append(('NB', GaussianNB(priors=None, var_smoothing=1e-08)))
models.append(('LSVM', LinearSVC(random_state=42, max_iter=1000, C=10)))
models.append(('ABDT', AdaBoostClassifier(DecisionTreeClassifier(random_state=42), n_estimators=5)))
models.append(('GB', GradientBoostingClassifier(random_state=42, max_depth=3)))
models.append(('MLP', MLPClassifier(random_state=42, solver='lbfgs', alpha=0.1, hidden_layer_sizes=(15,))))
models.append(('LDA', LinearDiscriminantAnalysis(solver='svd')))

In [ ]:
results = []
names = []
scores = []
stddevs = []
times = []

for name, model in models:
    score, stddev, elapsed = evaluate_model_cv(model, X=X, y=y)
    results.append((score, stddev))
    names.append(name)
    scores.append(score)
    stddevs.append(stddev)
    times.append(elapsed)

In [ ]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stddevs,
    'Time (ms)': times})

results_df.sort_values(by='Score', ascending=False)